In [87]:
from DbConnector import DbConnector
import pandas as pd
import numpy as np

connection = DbConnector()
db_connection = connection.db_connection
cursor = connection.cursor

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('assigment2_store_distribuerte',)
-----------------------------------------------



#### Task 2

In [55]:
"""
1. How many users, activities and trackpoints are there in the dataset (after it is
inserted into the database).
"""

num_users_query = """SELECT COUNT(*) FROM User"""
cursor.execute(num_users_query)
num_users = cursor.fetchone()[0]
print(f'Number of users in the dataset: {num_users}')

num_activities_query = """SELECT COUNT(*) FROM Activity"""
cursor.execute(num_activities_query)
num_activities = cursor.fetchone()[0]
print(f'Number of activities in the dataset: {num_activities}')

num_trackpoints_query = """SELECT COUNT(*) FROM TrackPoint"""
cursor.execute(num_trackpoints_query)
num_trackpoints = cursor.fetchone()[0]
print(f'Number of trackpoints in the dataset: {num_trackpoints}')

Number of users in the dataset: 182
Number of activities in the dataset: 16048
Number of trackpoints in the dataset: 9681756


In [57]:
"""
2. Find the average, maximum and minimum number of trackpoints per user.
"""

avg_max_min_per_user_query = """
SELECT AVG(trackpoint_count), MAX(trackpoint_count), MIN(trackpoint_count)
FROM (
    SELECT A.user_id AS user_id, COUNT(T.id) AS trackpoint_count
    FROM Activity A
    INNER JOIN TrackPoint T ON A.id = T.activity_id
    GROUP BY A.user_id
) AS UserTrackpointCounts
"""

cursor.execute(avg_max_min_per_user_query)
avg_max_min_per_user = cursor.fetchall()
print(f'Average number of trackpoints per user: {avg_max_min_per_user[0][0]}')
print(f'Max number of trackpoints per user: {avg_max_min_per_user[0][1]}')
print(f'Min number of trackpoints per user: {avg_max_min_per_user[0][2]}')

Average number of trackpoints per user: 55963.9075
Max number of trackpoints per user: 1010325
Min number of trackpoints per user: 17


In [160]:
"""
3. Find the top 15 users with the highest number of activities.
"""

top15_users_query = """
SELECT U.id, COUNT(A.id) AS ActivityCount
FROM User U
INNER JOIN Activity A ON U.id = A.user_id
GROUP BY U.id
ORDER BY ActivityCount DESC
LIMIT 15;
"""

cursor.execute(top15_users_query)
top15_users = cursor.fetchall()
for users in top15_users:
    print(f'User {users[0]}  \tActivity count {users[1]}')

User 128  	Activity count 2102
User 153  	Activity count 1793
User 25  	Activity count 715
User 163  	Activity count 704
User 62  	Activity count 691
User 144  	Activity count 563
User 41  	Activity count 399
User 85  	Activity count 364
User 4  	Activity count 346
User 140  	Activity count 345
User 167  	Activity count 320
User 68  	Activity count 280
User 17  	Activity count 265
User 3  	Activity count 261
User 14  	Activity count 236


In [50]:
"""
4. Find all users who have taken a bus.
"""

bus_users_query = """
SELECT U.id
FROM User U
INNER JOIN Activity A ON U.id = A.user_id
WHERE A.transportation_mode = "Bus"
GROUP BY U.id
"""

cursor.execute(bus_users_query)
bus_users = cursor.fetchall()
bussers = [user[0] for user in bus_users]
print(f'Bus users:')
for busser in bussers:
    print(busser)

Bus users:
10
52
62
73
81
84
85
91
92
112
125
128
175


In [58]:
"""
5. List the top 10 users by their amount of different transportation modes.
"""

nunique_transportation_mode_query = """
SELECT U.id, COUNT(DISTINCT A.transportation_mode) AS unique_transportation_mode
FROM User U
INNER JOIN Activity A on U.id = A.user_id
GROUP BY U.id
ORDER BY unique_transportation_mode DESC
LIMIT 10
"""

cursor.execute(nunique_transportation_mode_query)
nunique_transportation_mode = cursor.fetchall()

for users in nunique_transportation_mode:
    print(f'User {users[0]}  Transportation mode count {users[1]}')

User 128  Transportation mode count 10
User 62  Transportation mode count 8
User 85  Transportation mode count 5
User 84  Transportation mode count 4
User 163  Transportation mode count 4
User 112  Transportation mode count 4
User 81  Transportation mode count 4
User 78  Transportation mode count 4
User 58  Transportation mode count 4
User 102  Transportation mode count 3


In [163]:
"""
6. Find activities that are registered multiple times. You should find the query even
if it gives zero result.
"""

duplicate_activities_query = """
SELECT user_id, transportation_mode, start_date_time, end_date_time, COUNT(*) AS occurrence_count
FROM Activity
GROUP BY user_id, transportation_mode, start_date_time, end_date_time
HAVING COUNT(*) > 1;
"""

cursor.execute(duplicate_activities_query)
duplicate_activities = cursor.fetchall()
duplicate_activities

[]

In [74]:
"""
7. a) Find the number of users that have started an activity in one day and ended
the activity the next day.
"""

activity_time_query = """
SELECT A.user_id, A.id, A.transportation_mode, A.start_date_time, A.end_date_time
FROM User U
INNER JOIN Activity A ON U.id = A.user_id
WHERE DATE(A.start_date_time) + INTERVAL 1 DAY = DATE(A.end_date_time);
"""

df = pd.read_sql_query(activity_time_query, db_connection)

print(f'Number of users which had activities starting in one day, ending in the next day: {df["user_id"].nunique()}')

"""
7. b) List the transportation mode, user id and duration for these activities
"""

df["time_duration"] = df.end_date_time - df.start_date_time
df[['transportation_mode', 'user_id', 'time_duration']]

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_25012\1765195024.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(activity_time_query, db_connection)


Number of users which had activities starting in one day, ending in the next day: 98


,transportation_mode,user_id,time_duration
0,other,0,0 days 04:09:15
1,other,0,0 days 00:10:45
2,other,0,0 days 11:03:55
3,other,0,0 days 01:43:20
4,other,0,0 days 16:24:06
...,...,...,...
1006,other,94,0 days 00:48:17
1007,other,94,0 days 01:17:45
1008,other,94,0 days 01:19:02
1009,other,95,0 days 14:33:50


In [52]:
"""
8. Find the number of users which have been close to each other in time and space.
Close is defined as the same space (50 meters) and for the same half minute (30
seconds)
"""

proximity_query = """
SELECT A.user_id, A.id, T.date_time, T.lat, T.lon
FROM Activity A
INNER JOIN TrackPoint T ON A.id = T.activity_id
"""

proximity = pd.read_sql_query(proximity_query, db_connection)

proximity['user_id'] = proximity['user_id'].astype(int)

# Sort values based on date_time (timestamps) to get oreceeding user_ids under each other
proximity.sort_values(['date_time'], inplace=True)

# Calculate the Haversine distance between consecutive rows within the same user_id
def haversine(lat1, lon1, lat2, lon2):
    r = 6371000  # Earth radius in meters
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return r * c

# Callating haversine distance and time difference between each row
proximity['distance'] = haversine(proximity['lat'], proximity['lon'], proximity['lat'].shift(), proximity['lon'].shift())
proximity['time_diff'] = proximity.groupby('user_id')['date_time'].diff()

# Mask for when user_id changes from one value to the next
user_id_changed = proximity['user_id'] != proximity['user_id'].shift()

# Mask for time_diff from one row to the next where time is less than 30 seconds
time_diff_less_than_30s = proximity['time_diff'] <= pd.Timedelta(seconds=30)

# Mask for distance from onerow to the next where the distance is less than 30 seconds
distance_less_than_50m = proximity['distance'] <= 50

time_and_distance_mask = user_id_changed & distance_less_than_50m & time_diff_less_than_30s
time_and_distace = proximity[time_and_distance_mask]
print(f"Number of users that have been close in time and space: {time_and_distace['user_id'].nunique()}")

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_25012\1933607711.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  proximity = pd.read_sql_query(proximity_query, db_connection)


Number of users that have been close in time and space: 105


In [104]:
"""
9. Find the top 15 users who have gained the most altitude meters.
"""

altitude_gainers_query = """
SELECT A.user_id, SUM(gained_altitude_activity) * 0.3048 AS gained_altitude_total
FROM (
    SELECT T.activity_id AS activity_id,
        GREATEST(T.altitude - LAG(T.altitude) OVER (ORDER BY T.id), 0) AS gained_altitude_activity
    FROM TrackPoint T
    WHERE altitude != -777
) AS t
INNER JOIN Activity A ON t.activity_id = A.id
GROUP BY A.user_id
ORDER BY gained_altitude_total DESC
LIMIT 15;
"""

altitude_gainers = pd.read_sql_query(altitude_gainers_query, db_connection)
altitude_gainers

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_25012\1932969132.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  altitude_gainers = pd.read_sql_query(altitude_gainers_query, db_connection)


,user_id,gained_altitude_total
0,128,688280.7672
1,153,675336.5208
2,4,357835.5048
3,41,280401.9792
4,62,258156.1512
5,3,249477.2760
6,163,246217.7448
7,85,239366.4504
8,144,224671.4328
9,30,183820.0032


In [108]:
"""
10.Find the users that have traveled the longest total distance in one day for each
transportation mode.
"""

activity_time_query = """
SELECT A.user_id, A.id, A.transportation_mode, A.start_date_time, A.end_date_time, T.lat, T.lon
FROM Activity A
INNER JOIN TrackPoint T ON A.id = T.activity_id
WHERE DATE(A.start_date_time) = DATE(A.end_date_time);
"""

df = pd.read_sql_query(activity_time_query, db_connection)

df.sort_values(by=['user_id', 'id', 'start_date_time'], inplace=True)
grouped = df.groupby(['user_id', 'id', 'transportation_mode'])
result_df = grouped.apply(lambda group: haversine(
    group['lat'].iloc[0],
    group['lon'].iloc[0],
    group['lat'].iloc[-1],
    group['lon'].iloc[-1]
)).reset_index(name='distance')

max_distance_idx = result_df.groupby(['transportation_mode'])['distance'].idxmax()
max_distances = result_df.loc[max_distance_idx, ['user_id', 'transportation_mode', 'distance']].sort_values(by="distance", ascending=False)
max_distances['distance (km)'] = max_distances['distance'] / 1000
max_distances[['user_id', 'transportation_mode', 'distance (km)']]

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_25012\2001960017.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(activity_time_query, db_connection)


KeyboardInterrupt: 

In [109]:
df.sort_values(by=['user_id', 'id', 'start_date_time'], inplace=True)
grouped = df.groupby(['user_id', 'id', 'transportation_mode'])
result_df = grouped.apply(lambda group: group.apply(lambda row: haversine(
    row['lat'], row['lon'],
    group['lat'], group['lon']
).sum(), axis=1)).reset_index(name='total_distance')

# Find the user with the maximum total distance for each transportation mode
max_distance_idx = result_df.groupby(['transportation_mode'])['total_distance'].idxmax()
max_distances = result_df.loc[max_distance_idx, ['user_id', 'transportation_mode', 'total_distance']].sort_values(by="total_distance", ascending=False)
max_distances['total_distance (km)'] = max_distances['total_distance'] / 1000
max_distances[['user_id', 'transportation_mode', 'total_distance (km)']]


In [200]:
"""
11.Find all users who have invalid activities, and the number of invalid activities per
# user

An invalid activity is defined as an activity with consecutive trackpoints
 where the timestamps deviate with at least 5 minutes.
"""

invalid_query = """
SELECT user_id, num_invalid_activities, date_time, diff_seconds
FROM (
    SELECT A.user_id, A.id as num_invalid_activities, T.date_time,
        TIMESTAMPDIFF(SECOND, lag(T.date_time) OVER (PARTITION BY T.activity_id ORDER BY T.date_time), T.date_time) AS diff_seconds
    FROM Activity A
    INNER JOIN TrackPoint T ON A.id = T.activity_id
) AS subquery
WHERE diff_seconds > 300;
"""

invalid = pd.read_sql_query(invalid_query, db_connection)
invalid['user_id'] = invalid['user_id'].astype(int)
invalid = invalid.sort_values(by='user_id')
display(invalid.groupby(['user_id'])['num_invalid_activities'].nunique().reset_index())

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_8716\3510112159.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  invalid = pd.read_sql_query(invalid_query, db_connection)


,user_id,num_invalid_activities
0,0,101
1,1,45
2,2,98
3,3,179
4,4,219
...,...,...
166,175,4
167,176,8
168,179,28
169,180,2


In [253]:
"""
12.Find all users who have registered transportation_mode and their most used
transportation_mode.
    - The answer should be on format (user_id,most_used_transportation_mode) sorted on user_id.
    - Some users may have the same number of activities tagged with e.g. walk 
    and car. In this case it is up to you to decide which transportation mode
    to include in your answer (choose one)
    - Do not count the rows where the mode is null.
"""

transportation_mode_query = """
SELECT user_id, most_used_transportation_mode, num_mode
FROM (
    SELECT 
        user_id, 
        transportation_mode AS most_used_transportation_mode, 
        COUNT(id) AS num_mode,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY COUNT(id) DESC) AS rnk
    FROM Activity
    WHERE transportation_mode != 'other'
    GROUP BY user_id, transportation_mode
) AS ranked_activities
WHERE rnk = 1
"""

transportation_mode = pd.read_sql_query(transportation_mode_query, db_connection)
transportation_mode['user_id'] = transportation_mode['user_id'].astype(int)
transportation_mode = transportation_mode.sort_values(by='user_id')


pd.set_option('display.max_rows', 20)
display(transportation_mode)
pd.reset_option('display.max_rows')

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_8716\521215358.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transportation_mode = pd.read_sql_query(transportation_mode_query, db_connection)


,user_id,most_used_transportation_mode,num_mode
0,10,taxi,3
21,20,bike,79
22,21,walk,1
23,52,bus,1
24,56,bike,15
...,...,...,...
16,153,walk,5
17,161,walk,1
18,163,bike,26
19,167,bike,31
